In [ ]:
import numpy as np
import math
import random


modulo = 59
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?&@"

key_matrix = np.array([
    [3, 2],
    [5, 7]
])

text1 = open('text/one.txt').read()
text2 = open('text/two.txt').read()
text3 = open('text/three.txt').read()
text4 = open('text/four.txt').read()

In [ ]:
def generateKeyMatrix(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        # Compute determinant mod
        a, b = key_matrix[0]
        c, d = key_matrix[1]
        det = (a*d - b*c) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")


def generate_key_matrix_nxn(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        # Compute determinant mod
        det = int(round(np.linalg.det(key_matrix))) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            # Not invertible, try again
            pass


def inverse_2x2_mod(matrix, modulo):
    # Compute modular inverse for a 2x2 matrix using integer arithmetic only
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    det_inv = pow(det, -1, modulo)
    # Adjugate matrix
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo


def inverse_nxn_mod(matrix, modulo):
    # Convert to integers and work on a copy
    A = np.array(matrix, dtype=int) % modulo
    n = A.shape[0]

    # Create identity matrix
    I = np.eye(n, dtype=int)

    # Perform Gaussian elimination
    for i in range(n):
        # Find pivot (row with non-zero element in column i)
        for r in range(i, n):
            if A[r, i] % modulo != 0:
                # Swap pivot row if necessary
                if r != i:
                    A[[i, r]] = A[[r, i]]
                    I[[i, r]] = I[[r, i]]
                break
        else:
            # No pivot found, matrix not invertible
            raise ValueError("Matrix not invertible modulo {}".format(modulo))

        # Normalize pivot row
        inv_pivot = pow(int(A[i, i]), -1, modulo)
        A[i] = (A[i] * inv_pivot) % modulo
        I[i] = (I[i] * inv_pivot) % modulo

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r, i] % modulo
                if factor != 0:
                    A[r] = (A[r] - factor * A[i]) % modulo
                    I[r] = (I[r] - factor * I[i]) % modulo

    return I % modulo


def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range
    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]
    # Pad if necessary
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T
    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix


def encryptNN(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    n = key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range
    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]
    # Pad if necessary
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T
    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix


def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    cipher_matrix = np.array(numbers).reshape(-1, 2)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    return ''.join(plaintext_chars)


def decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    dec = ''.join(plaintext_chars)
    return dec.rstrip('A')


def generate_random_string(alphabet, l):
    return ''.join(random.choice(alphabet) for _ in range(l))


def test_encryption_decryption(l):
    plaintext = generate_random_string(alphabet, l)
    ciphertext, ciphertext_matrix = encrypt(plaintext, key_matrix, modulo, alphabet)
    decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
    return plaintext, decrypted_text, plaintext == decrypted_text

In [ ]:
inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)

for idx in range(1, 51):
    plaintext, decrypted_text, result = test_encryption_decryption(200)
    if result:
        print(f"Test {idx}: True")
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f"  Decrypted: {decrypted_text}")

In [ ]:
inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(text1, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", text1)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")
print(f"Condition: {np.linalg.cond(key_matrix)}")


In [ ]:
key_matrix, inverse_key_matrix = generateKeyMatrix(2, modulo)

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(text4, key_matrix, modulo, alphabet)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", text4)
print()
print(f"Ciphertext: {ciphertext}")
print()
print(f"Decrypted Text: {decrypted_text}")
print()
print(f"Condition: {np.linalg.cond(key_matrix)}")

In [ ]:
keyMatrixSize = 4
key_matrix, inverse_key_matrix = generate_key_matrix_nxn(keyMatrixSize, modulo)

ciphertext, ciphertext_matrix = encryptNN(text1, key_matrix, modulo, alphabet)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", text1)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

In [ ]:
keyMatrixSize = 7
key_matrix, inverse_key_matrix = generate_key_matrix_nxn(keyMatrixSize, modulo)

ciphertext, ciphertext_matrix = encryptNN(text1, key_matrix, modulo, alphabet)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", text1)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

In [ ]:
keyMatrixSize = 10
key_matrix, inverse_key_matrix = generate_key_matrix_nxn(keyMatrixSize, modulo)

ciphertext, ciphertext_matrix = encryptNN(text1, key_matrix, modulo, alphabet)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decryptNN(ciphertext, inverse_key_matrix, modulo, alphabet)
print("Original Text:", text1)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")